# GPT4oを用いた画像処理について試す

## 概要

GPT4oを用いて画像処理を行う方法を試し、どのような活用方法が考えられるかを考察する。

## 事前準備

1. 以下公式よりAzureOpenAIリソースを作成し、GPT4oのモデルをデプロイし使用可能な状態とする。
    - [Azure OpenAIリソースの作成](https://learn.microsoft.com/ja-jp/azure/ai-services/openai/how-to/create-resource?pivots=web-portal)
2. .envファイルを作成する。

    ```
    cp .devcontainer/.env.example .devcontainer/.env
    ```

3. .envファイルの下記項目に適切な値を記述する。

    ```
    OPENAI_API_KEY=xxxxxxxx
    OPENAI_API_BASE=https://{resource name}.openai.azure.com/
    OPENAI_API_VERSION=2024-02-01
    OPENAI_DEPLOYMENT_NAME_GPT4o={deployment name}
    ```

4. Devcontainerを起動する。

## Install

In [1]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


## Environment

In [2]:
import os

AZURE_OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT = os.environ.get('OPENAI_API_BASE')
DEPLOYMENT_NAME = os.environ.get('OPENAI_DEPLOYMENT_NAME_GPT4o')
API_VERSION = os.environ.get('OPENAI_API_VERSION')

## Usage

参考：[公式サンプルコード](https://learn.microsoft.com/ja-jp/azure/ai-services/openai/gpt-v-quickstart?tabs=image%2Ccommand-line&pivots=programming-language-python#create-a-new-python-application)

In [27]:
from openai import AzureOpenAI

def gptImageAnalysis(userMessage :str, imageUrl :str):
    client = AzureOpenAI(
        api_key=AZURE_OPENAI_API_KEY,  
        api_version=API_VERSION,
        base_url=f"{AZURE_OPENAI_ENDPOINT}/openai/deployments/{DEPLOYMENT_NAME}"
    )

    response = client.chat.completions.create(
        model=DEPLOYMENT_NAME,
        messages=[
            { "role": "system", "content": "You are a helpful assistant. Describe this picture. Please output in Japanese." },
            { "role": "user", "content": [  
                { 
                    "type": "text", 
                    "text": userMessage 
                },
                { 
                    "type": "image_url",
                    "image_url": {
                        "url": imageUrl
                    }
                }
            ] } 
        ],
        # max_tokens=2000 　GPT4oはデフォルト4096の為指定不要
    )

    return response

def getOutputMessage(response):
    return response.choices[0].message.content

試しに上記の公式サンプルコードにあった画像を使用  
![](https://learn.microsoft.com/ja-jp/azure/ai-services/openai/media/quickstarts/endpoint.png#lightbox)

In [29]:
message = ""
url = 'https://learn.microsoft.com/ja-jp/azure/ai-services/openai/media/quickstarts/endpoint.png#lightbox'
response = gptImageAnalysis(message, url)

print(getOutputMessage(response))

この画像は、Microsoft Azure ポータルの「Keys and Endpoint」セクションを示しています。具体的には、「docs-test-001」という名前のCognitive Serviceリソースのキーとエンドポイントの情報が表示されています。画面には次の情報が含まれています。

- 左側のナビゲーションメニューには、「Keys and Endpoint」が選択されている。
- 右側の主要なセクションには、「KEY 1」と「KEY 2」としてそれぞれのキーが隠されています。
- 「Location/Region」として「eastus」が表示されています。
- 「Endpoint」としてURL（「https://docs-test-001.openai.azure.com/」）が表示されています。
- また、「Show Keys」ボタンや、「Regenerate Key1」「Regenerate Key2」のボタンも表示されています。

その他のリソース管理オプションとしては、「Overview」、「Activity log」、「Access control (IAM)」、「Tags」、「Deployments」、「Pricing tier」、「Networking」、「Identity」、「Cost analysis」、「Properties」、「Locks」が左側のメニューにアイコンと共にリストされています。


#### URLだけではなく、画像ファイルをBase64に変換し処理することも可能

In [25]:
import urllib.request
import base64

# ImageFileをBase64エンコード
def convertImageFileToBase64data(file_path):
    # base64 エンコード
    with open(file_path, 'rb') as image:
        base64_data = base64.b64encode(image.read()).decode('utf-8')

    return f"data:image/jpg;base64,{base64_data}"

# ImageURLをBase64エンコード
def convertImageURLToBase64data(url):
    # 画像ファイルダウンロード
    dst_path = '../data/temp/temp.png'
    urllib.request.urlretrieve(url, dst_path)

    return convertImageFileToBase64data(dst_path)


In [45]:
url = 'https://learn.microsoft.com/ja-jp/azure/ai-services/openai/media/quickstarts/endpoint.png#lightbox'

base64_image_data = convertImageURLToBase64data(url)
response = gptImageAnalysis(message, base64_image_data)

print(getOutputMessage(response))

この画像は、Microsoft Azureのポータルで表示されているCognitive Serviceの「Keys and Endpoint」ページのスクリーンショットです。このページではAPIのキーやエンドポイントに関する情報が提供されています。

1. **APIキー（KEY 1, KEY 2）**:
   - それぞれのキーは、Azure Cognitive Service APIにアクセスするために必要な認証情報です。
   - これらのキーを使用することで、APIリクエストを行う際に認証を行います。
   - キーは定期的に再生成することが推奨されています。

2. **エンドポイント（Endpoint）**:
   - APIへのリクエストを送信するためのURLです。
   - この例では、エンドポイントURLは「https://docs-test-001.openai.azure.com/」です。
   - リージョンが「eastus」と設定されています。この情報は、Azureが地理的に最適なデータセンターを選ぶために使用します。

### データの流れ

1. **リクエストの準備**:
   - クライアントアプリケーションは、利用するCognitive Service APIのエンドポイントURLと認証キー（例：KEY 1またはKEY 2）を設定します。

2. **APIリクエストの送信**:
   - クライアントアプリケーションは、エンドポイントURLに向けてHTTPリクエストを送信します。このリクエストには、必要なパラメータやデータが含まれます。
   - リクエストヘッダーの一部として、認証キーが含まれます。

3. **認証と処理**:
   - サービスが認証キーを確認し、リクエストが正当なものであるかを確認します。
   - 認証後、サービスはリクエストの内容に従って処理を行います。

4. **レスポンスの受信**:
   - 処理が完了すると、Cognitive Service APIはクライアントアプリケーションに対して応答を送信します。
   - 応答には、リクエストされたデータや情報、または処理結果が含まれます。

このプロセスを通じて、クライアントとサーバー間でデータが安全にやり取りされます。


## Example

### 01.グラフ情報の読み取り

![](../data/picture/グラフ.jpg)

出典：「生成AIの世界需用額見通し」より、電子情報技術産業協会（JEITA）、2023年12月21日  
https://www.jeita.or.jp/cgi-bin/topics/detail.cgi?n=4724

In [26]:
file_path = '../data/picture/グラフ.jpg'

base64_image_data = convertImageFileToBase64data(file_path)
response = gptImageAnalysis(message, base64_image_data)

print(getOutputMessage(response))

この画像には、生成AI市場の需要額（日本）についてのグラフが示されています。以下の説明が含まれています：

1. **タイトル**：生成AI市場の需要額（日本）
2. **年ごとの需要額**：
   - 2023年：1,188億円
   - 2025年：6,879億円
   - 2030年：17,774億円
3. **増加率**：
   - 年平均：47.2%増
   - 年平均の詳細：
     - 生成AI基盤モデル：46.0%増（黄色）
     - 生成AI関連アプリケーション：46.6%増（青色）
     - 生成AI関連ソリューションサービス：52.2%増（赤色）
4. **出典**：「生成AIの世界需用展望」より、電子情報技術産業協会（JEITA）、2023年12月21日

グラフは棒グラフ形式で、2023年から2030年にかけての生成AI市場の需用額の増加を視覚的に表しています。また、観衆がスライドを見ている様子が写っています。


In [30]:
file_path = '../data/picture/グラフ.jpg'
message = '年ごとの需要額は2023年から2023年でいくら増えましたか？'

base64_image_data = convertImageFileToBase64data(file_path)
response = gptImageAnalysis(message, base64_image_data)

print(getOutputMessage(response))

2023年から2030年で生成AI市場の需要額は増えました。2023年の需要額は1,188億円で、2030年の需要額は17,774億円です。その差を計算すると、17,774億円 - 1,188億円 =16,586億円増加しました。


### 02.領収書

![](../data/picture/領収書.jpg)

In [32]:
file_path = '../data/picture/領収書.jpg'
message = '日付と金額を教えて下さい'

base64_image_data = convertImageFileToBase64data(file_path)
response = gptImageAnalysis(message, base64_image_data)

print(getOutputMessage(response))

日付は2024年4月13日、金額は28,840円です。


### 03. Azure アーキテクチャの読み取り

![](../data/picture/azure_aks_architectures.jpeg)

出典: Azure Kubernetes Service 上のマイクロサービス アーキテクチャ  
https://learn.microsoft.com/ja-jp/azure/architecture/reference-architectures/containers/aks-microservices/aks-microservices

In [51]:
file_path = '../data/picture/azure_aks_architectures.jpeg'
message = 'このアーキテクチャを説明して、それぞれのデータの流れで説明して。'

base64_image_data = convertImageFileToBase64data(file_path)
response = gptImageAnalysis(message, base64_image_data)

print(getOutputMessage(response))

このアーキテクチャ図は、Microsoft Azureを使用したアプリケーションのデプロイメントおよび運用の流れを示しています。以下に各コンポーネントとデータフローの詳細を説明します。

### コンポーネントの説明：

1. **クライアントアプリ (Client Apps)**：
   クライアントはアプリケーションにアクセスします。

2. **Azure Load Balancer**：
   クライアントからのトラフィックを受け取り、Kubernetesクラスターのフロントエンドへ流します。

3. **Azure Kubernetes Service (AKS)**：
   Kubernetesクラスターをホスティングし、アプリケーションのスケーリングと管理を行います。

4. **Front End**（Namespace）：
   - **Ingress**：トラフィックを受け取り、バックエンドサービスにルーティングします。

5. **Back-End Services** （Namespace）：
   - ここにはアプリケーションの主要なビジネスロジックが含まれ、Podが自動的にスケールされます。
   - **SQL Database**：データの保管と管理。
   - **Azure Cosmos DB**：グローバルに分散されたデータベース。

6. **Utility Services** （Namespace）：
   - **Elasticsearch**：ログやデータのインデックスを検索可能にする。
   - **Prometheus**：モニタリングとアラートのためのデータ収集。

7. **CI/CD**：
   - **Azure Pipelines**：継続的インテグレーションおよびデリバリー。
   - **Helm Upgrade**：Kubernetesアプリケーションの管理。

8. **Azure Container Registry**：
   コンテナイメージのストレージ。

9. **Dev/Ops**：
   開発者や運用担当者が操作を行う。

10. **Azure Active Directory**：
    役割に基づくアクセス管理を提供。

11. **Azure Monitor**：
    

### 04. バーコード読み取り

![](../data/picture/barcode_.png)

元：漫画ワンピース108巻のバーコード

In [55]:
file_path = '../data/picture/barcode_.png'
message = '画像のバーコードのデータは何ですか？'

base64_image_data = convertImageFileToBase64data(file_path)
response = gptImageAnalysis(message, base64_image_data)

print(getOutputMessage(response))

申し訳ございませんが、画像のバーコードのデータを読み取ることはできません。詳細なバーコードデータを取得する必要がある場合は、バーコードリーダーまたは専用のアプリケーションを使用してみてください。


### 98. カロリー計算

![](../data/picture/ピザ.jpg)

In [34]:
file_path = '../data/picture/ピザ.jpg'
message = '画像の説明と推定カロリーを教えて下さい'

base64_image_data = convertImageFileToBase64data(file_path)
response = gptImageAnalysis(message, base64_image_data)

print(getOutputMessage(response))

この画像には、ピザが写っています。ピザはトマトソース、モッツァレラチーズ、ソーセージ、ベーコン、およびバジルがトッピングされています。生地の縁は程よく焼き色がつき、カリッとしていそうです。

推定カロリーは、ピザの種類やトッピングの量によりますが、一般的なマルゲリータピザサイズのピザとして推定すると、約600～800キロカロリー程度です。トッピングの量が多いため、800キロカロリーを超える可能性もあります。


![](../data/picture/スープカレー.jpg)

In [35]:
file_path = '../data/picture/スープカレー.jpg'
message = '画像の説明と推定カロリーを教えて下さい'

base64_image_data = convertImageFileToBase64data(file_path)
response = gptImageAnalysis(message, base64_image_data)

print(getOutputMessage(response))

この画像は、スープカレーの一品を示しています。スープカレーは、具材が豊富で、スパイシーなスープと一緒に提供されるカレー料理です。画像には以下の具材が含まれています：
- 人参
- ピーマン
- オクラ
- ブロッコリー
- アスパラガス
- れんこん
- かぼちゃ
- ナス
- 半熟卵
- カレーライス

推定カロリーは以下の通りです：
- 野菜や一部の具材：約150-200 kcal
- スープカレー：約300-400 kcal
- ライス：約200-250 kcal

合計すると、1食分の推定カロリーは約650-850 kcal程度になると考えられます。しかし、具材や調理方法によってカロリーは変動するため、あくまで推定値です。


### 99．その他

![](../data/picture/ドリンク_1.jpg)

In [42]:
file_path = '../data/picture/ドリンク_1.jpg'
message = """画像に最も大きく映っている液体が入っているグラスの液体の容量の割合を、以下のJSON形式でおしえてください。
{'description':'xxxxxxxxx', 'remainingamount':'xx%'}"""

base64_image_data = convertImageFileToBase64data(file_path)
response = gptImageAnalysis(message, base64_image_data)

print(getOutputMessage(response))

```json
{
  "description": "SAPPOROのロゴが入った大きなグラスに液体が入っている",
  "remainingamount": "80%"
}
```
